<a href="https://colab.research.google.com/github/kurianbenoy/MTTS/blob/master/Transliteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text-to-Speech with Mozilla Tacotron+WaveRNN

This is an English female voice TTS demo using open source projects [mozilla/TTS](https://github.com/mozilla/TTS/) and [erogol/WaveRNN](https://github.com/erogol/WaveRNN).

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).

## Install Mozilla TTS and WaveRNN

In [1]:
import os
import time
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/mozilla/TTS.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  !git clone -q {git_repo_url}
  !cd {project_name} && git checkout Tacotron2-iter-260K-824c091
  !pip install -q gdown lws librosa Unidecode==0.4.20 tensorboardX git+git://github.com/bootphon/phonemizer@master localimport
  !apt-get install -y espeak
git_repo_url = 'https://github.com/erogol/WaveRNN.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  !git clone -q {git_repo_url}
  !cd {project_name} && git checkout 8a1c152 && pip install -q -r requirements.txt

  
import sys
sys.path.append('TTS')
sys.path.append('WaveRNN')
from localimport import localimport
  
from IPython.display import Audio, display

Branch 'Tacotron2-iter-260K-824c091' set up to track remote branch 'Tacotron2-iter-260K-824c091' from 'origin'.
Switched to a new branch 'Tacotron2-iter-260K-824c091'
     |████████████████████████████████| 153kB 6.2MB/s 
     |████████████████████████████████| 235kB 16.6MB/s 
     |████████████████████████████████| 204kB 21.6MB/s 
     |████████████████████████████████| 194kB 22.0MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 32 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-

## Download pretrained models

In [2]:
# WaveRNN
!mkdir -p wavernn_models tts_models
wavernn_pretrained_model = 'wavernn_models/checkpoint_433000.pth.tar'
if not exists(wavernn_pretrained_model):
  !gdown -O {wavernn_pretrained_model} https://drive.google.com/uc?id=12GRFk5mcTDXqAdO5mR81E-DpTk8v2YS9
wavernn_pretrained_model_config = 'wavernn_models/config.json'
if not exists(wavernn_pretrained_model_config):
  !gdown -O {wavernn_pretrained_model_config} https://drive.google.com/uc?id=1kiAGjq83wM3POG736GoyWOOcqwXhBulv
    
# TTS
tts_pretrained_model = 'tts_models/checkpoint_261000.pth.tar'
if not exists(tts_pretrained_model):
  !gdown -O {tts_pretrained_model} https://drive.google.com/uc?id=1otOqpixEsHf7SbOZIcttv3O7pG0EadDx
tts_pretrained_model_config = 'tts_models/config.json'
if not exists(tts_pretrained_model_config):
  !gdown -O {tts_pretrained_model_config} https://drive.google.com/uc?id=1IJaGo0BdMQjbnCcOL4fPOieOEWMOsXE-

Downloading...
From: https://drive.google.com/uc?id=12GRFk5mcTDXqAdO5mR81E-DpTk8v2YS9
To: /content/wavernn_models/checkpoint_433000.pth.tar
50.9MB [00:00, 66.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kiAGjq83wM3POG736GoyWOOcqwXhBulv
To: /content/wavernn_models/config.json
100% 2.31k/2.31k [00:00<00:00, 5.45MB/s]
Downloading...
From: https://drive.google.com/uc?id=1otOqpixEsHf7SbOZIcttv3O7pG0EadDx
To: /content/tts_models/checkpoint_261000.pth.tar
338MB [00:02, 140MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IJaGo0BdMQjbnCcOL4fPOieOEWMOsXE-
To: /content/tts_models/config.json
100% 5.75k/5.75k [00:00<00:00, 4.82MB/s]


## Initialize models

In [3]:
#
# this code is copied from: https://github.com/mozilla/TTS/blob/master/notebooks/Benchmark.ipynb
#

import io
import torch 
import time
import numpy as np
from collections import OrderedDict
from matplotlib import pylab as plt
import IPython

%pylab inline
rcParams["figure.figsize"] = (16,5)

import librosa
import librosa.display

from TTS.models.tacotron import Tacotron 
from TTS.layers import *
from TTS.utils.data import *
from TTS.utils.audio import AudioProcessor
from TTS.utils.generic_utils import load_config, setup_model
from TTS.utils.text import text_to_sequence
from TTS.utils.synthesis import synthesis
from TTS.utils.visual import visualize

def tts(model, text, CONFIG, use_cuda, ap, use_gl, speaker_id=None, figures=True):
    t_1 = time.time()
    waveform, alignment, mel_spec, mel_postnet_spec, stop_tokens = synthesis(model, text, CONFIG, use_cuda, ap, truncated=True, enable_eos_bos_chars=CONFIG.enable_eos_bos_chars)
    if CONFIG.model == "Tacotron" and not use_gl:
        mel_postnet_spec = ap.out_linear_to_mel(mel_postnet_spec.T).T
    if not use_gl:
        waveform = wavernn.generate(torch.FloatTensor(mel_postnet_spec.T).unsqueeze(0).cuda(), batched=batched_wavernn, target=11000, overlap=550)

    print(" >  Run-time: {}".format(time.time() - t_1))
    if figures:                                                                                                         
        visualize(alignment, mel_postnet_spec, stop_tokens, text, ap.hop_length, CONFIG, mel_spec)                                                                       
    IPython.display.display(Audio(waveform, rate=CONFIG.audio['sample_rate']))  
    #os.makedirs(OUT_FOLDER, exist_ok=True)
    #file_name = text.replace(" ", "_").replace(".","") + ".wav"
    #out_path = os.path.join(OUT_FOLDER, file_name)
    #ap.save_wav(waveform, out_path)
    return alignment, mel_postnet_spec, stop_tokens, waveform
  
use_cuda = True
batched_wavernn = True

# initialize TTS
CONFIG = load_config(tts_pretrained_model_config)
from TTS.utils.text.symbols import symbols, phonemes
# load the model
num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, CONFIG)
# load the audio processor
ap = AudioProcessor(**CONFIG.audio)         
# load model state
if use_cuda:
    cp = torch.load(tts_pretrained_model)
else:
    cp = torch.load(tts_pretrained_model, map_location=lambda storage, loc: storage)

# load the model
model.load_state_dict(cp['model'])
if use_cuda:
    model.cuda()
model.eval()
print(cp['step'])
model.decoder.max_decoder_steps = 2000

# initialize WaveRNN
VOCODER_CONFIG = load_config(wavernn_pretrained_model_config)
with localimport('/content/WaveRNN') as _importer:
  from models.wavernn import Model
bits = 10

wavernn = Model(
        rnn_dims=512,
        fc_dims=512,
        mode="mold",
        pad=2,
        upsample_factors=VOCODER_CONFIG.upsample_factors,  # set this depending on dataset
        feat_dims=VOCODER_CONFIG.audio["num_mels"],
        compute_dims=128,
        res_out_dims=128,
        res_blocks=10,
        hop_length=ap.hop_length,
        sample_rate=ap.sample_rate,
    ).cuda()
check = torch.load(wavernn_pretrained_model)
wavernn.load_state_dict(check['model'])
if use_cuda:
    wavernn.cuda()
wavernn.eval()
print(check['step'])

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > bits:None
 | > sample_rate:22050
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:12.5
 | > frame_length_ms:50
 | > ref_level_db:20
 | > num_freq:1025
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:False
 | > mel_fmin:0.0
 | > mel_fmax:8000.0
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > n_fft:2048
 | > hop_length:275
 | > win_length:1102
261000
433000


## Sentence to synthesize

In [0]:
SENTENCE = 'Bill got in the habit of asking himself “Is that thought true?” And if he wasn’t absolutely certain it was, he just let it go.'

## Synthetize

In [5]:
align, spec, stop_tokens, wav = tts(model, SENTENCE, CONFIG, use_cuda, ap, speaker_id=0, use_gl=False, figures=False)

180000/181500 -- batch_size: 15 -- gen_rate: 9.6 kHz -- x_realtime: 0.4   >  Run-time: 29.55617094039917


In [0]:
subin = '3, 2,1 and go.The biggest news in 2019 has to be Janayugoms move to Scribus this is a landmark event since this is the first time an entirely free software based stack is used to produce a newspaper in Malayalam'

In [7]:
%%time
align, spec, stop_tokens, wav = tts(model, subin, CONFIG, use_cuda, ap, speaker_id=1, use_gl=False, figures=False)

   | > Decoder stopped with 'max_decoder_steps
576000/580800 -- batch_size: 48 -- gen_rate: 30.4 kHz -- x_realtime: 1.4   >  Run-time: 49.380247354507446


CPU times: user 48.8 s, sys: 709 ms, total: 49.6 s
Wall time: 49.6 s


## Transliterated text

Thanks to subins https://demos.subinsb.com/indicen/

In [0]:
text1 = "vandi itukki etthi"

In [9]:
print("Hello")

Hello


In [11]:
%time
align, spec, stop_tokens, wav = tts(model, text1, CONFIG, use_cuda, ap, speaker_id=1, use_gl=False, figures=False)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
36000/36300 -- batch_size: 3 -- gen_rate: 1.9 kHz -- x_realtime: 0.1   >  Run-time: 21.583120107650757


In [0]:
text2 = "njaan innu oru vandi kayari"

In [13]:
%time
align, spec, stop_tokens, wav = tts(model, text2, CONFIG, use_cuda, ap, speaker_id=1, use_gl=False, figures=False)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
60000/60500 -- batch_size: 5 -- gen_rate: 3.1 kHz -- x_realtime: 0.1   >  Run-time: 22.931612253189087


In [0]:
text3 = "athil niraye aalukal undaayirunnu"

In [16]:
%time
align, spec, stop_tokens, wav = tts(model, text3, CONFIG, use_cuda, ap, speaker_id=1, use_gl=False, figures=False)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs
60000/60500 -- batch_size: 5 -- gen_rate: 3.2 kHz -- x_realtime: 0.1   >  Run-time: 22.982138872146606
